###SVM

In [3]:
import graphlab as gl
login = gl.SFrame.read_csv('data/user_login_info.csv')
eng = gl.SFrame.read_csv('data/user_engagement.csv')
login_eng = login.join(eng, on='user_id', how='inner')

PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_login_info.csv
PROGRESS: Parsing completed. Parsed 100 lines in 3.81788 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 3172097 lines. Lines per second: 1.02576e+06
PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_login_info.csv
PROGRESS: Parsing completed. Parsed 4000000 lines in 3.4383 secs.
PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_engagement.csv
PROGRESS: Parsing completed. Parsed 100 lines in 1.47201 secs.
------------------------------------------------

###Regular SVM

In [9]:
features=[
    'num_tweets_30d',
    'num_tweet_days_30d',
    'time_in_app_30d',
    'num_timeline_views_30d',
    'num_share_sent_days_30d',
    'num_share_rcvd_30d',
    'num_favour_sent_30d',
    'num_favour_rcvd_30d',
    ]

train_login_eng, test_login_eng = login_eng.random_split(0.8, seed=1)

In [10]:
svm = gl.svm_classifier.create(train_login_eng,
                               target='activity',
                               features=features,
                               max_iterations=10)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: SVM:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 3040056
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 8
PROGRESS: Number of unpacked features : 8
PROGRESS: Number of coefficients    : 9
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 3        | 0.000000  | 1.330255     | 0.803826          | 0.803193            |
PROGRESS: | 2         | 

In [12]:
#test
res = svm.evaluate(test_login_eng)
print res

{'confusion_matrix': Columns:
	target_label	int
	predicted_label	int
	count	int

Rows: 4

Data:
+--------------+-----------------+--------+
| target_label | predicted_label | count  |
+--------------+-----------------+--------+
|      0       |        1        | 28441  |
|      0       |        0        | 128013 |
|      1       |        0        | 86465  |
|      1       |        1        | 556787 |
+--------------+-----------------+--------+
[4 rows x 3 columns]
, 'accuracy': 0.8563146956506517}


SVM gives a 0.85 accuracy. Next, we will try to better the accuracy by some feature engineering. Quadratic features functions generates the product of all feature combinations and treats them as new features. 
For example, if there are 3 features 'a'=[1,2], 'b'=[3,4], 'c'=[5,6] then quadratic features will generate 3 more features 'ab'=[3,8],

####Quadratic features

In [13]:
# transform
quad = gl.feature_engineering.QuadraticFeatures()
train_quad = quad.fit_transform(train_login_eng)
test_quad = quad.fit_transform(test_login_eng)

In [14]:
training_features = features.append('quadratic_features')
svm_quad = gl.svm_classifier.create(train_quad,
                                     target='activity',
                                     features=training_features,
                                     max_iterations=10)
print svm_quad

res_quad = svm_quad.evaluate(test_quad)
print res_quad

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: SVM:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 3040580
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 13
PROGRESS: Number of unpacked features : 298
PROGRESS: Number of coefficients    : 308
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 3        | 0.000000  | 35.295607    | 0.824576          | 0.826158            |
PROGRESS: | 2      

The svm classifier fits the data with 100% accuracy. This could mean its overfitting.